In [42]:
import pyreadr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import chi2_contingency

In [43]:
data_2014 = pyreadr.read_r('cleaned_data/data14.rds')
df_14=data_2014[None]

data_2015_18 = pyreadr.read_r('cleaned_data/data1518.rds')
df_1518=data_2015_18[None]

data_2011_14 = pyreadr.read_r('cleaned_data/new_data1114.rds')
df_1114=data_2011_14[None]

new_data_2015_18 = pyreadr.read_r('cleaned_data/new_data1518.rds')
df_new_1518=new_data_2015_18[None]

'''
with pd.ExcelWriter('output2.xlsx') as writer:
    df_1114.to_excel(writer, sheet_name='Sheet_name_1')
'''

"\nwith pd.ExcelWriter('output2.xlsx') as writer:\n    df_1114.to_excel(writer, sheet_name='Sheet_name_1')\n"

In [44]:
titles=df_14.keys()
titles2 = df_1518.keys()
print(titles)
print(titles2)

titles3=df_1114.keys()
titles4 = df_new_1518.keys()
print(titles3)
print(titles4)

Index(['xstate', 'imonth', 'genhlth', 'physhlth', 'poorhlth', 'hlthpln1',
       'persdoc2', 'medcost', 'checkup1', 'exerany2', 'sex', 'educa',
       'employ1', 'income2', 'flushot6', 'howlong', 'lastpap2', 'lastsig3',
       'hadsigm3', 'xhcvu651', 'xtotinda', 'drvisits', 'medicare', 'hlthcvr1',
       'nocov121', 'medscost', 'carercvd', 'year'],
      dtype='object')
Index(['xstate', 'imonth', 'genhlth', 'physhlth', 'poorhlth', 'hlthpln1',
       'persdoc2', 'medcost', 'checkup1', 'exerany2', 'sex', 'educa',
       'employ1', 'income2', 'flushot6', 'howlong', 'lastpap2', 'lastsig3',
       'hadsigm3', 'xhcvu651', 'xtotinda', 'year', 'drvisits', 'medicare',
       'hlthcvr1', 'nocov121', 'medscost', 'carercvd'],
      dtype='object')
Index(['xstate', 'genhlth', 'hlthpln1', 'numadult', 'race', 'income2',
       'physhlth', 'poorhlth', 'persdoc2', 'checkup1', 'exerany2', 'sex',
       'educa', 'numadult.1', 'children', 'employ', 'flushot5', 'howlong',
       'lastpap2', 'lastsig3', 'ha

In [45]:
low_income14 = df_14.loc[df_14['income2'] <= 4]
low_income1518 = df_1518.loc[df_1518['income2'] <= 4]

In [46]:
#function to return the counts of people for each category in a given column
def getCounts(df, num_cols, threshold = 7):
    counts = [0 for i in range(num_cols)]
    for row in df:
        value = (int)(row)
        if value < threshold:
            counts[value-1] += 1
    return counts

In [47]:
#start of chi2 test for general health data
gen_health14 = df_14['genhlth']
gen_health14

0        1.0
1        3.0
2        2.0
3        1.0
4        3.0
        ... 
51853    3.0
51854    2.0
51855    2.0
51856    1.0
51857    2.0
Name: genhlth, Length: 51858, dtype: float64

In [48]:
gen_health1518 = df_1518['genhlth']
gen_health1518

0         3.0
1         3.0
2         3.0
3         1.0
4         3.0
         ... 
193644    3.0
193645    3.0
193646    1.0
193647    2.0
193648    1.0
Name: genhlth, Length: 193649, dtype: float64

In [49]:
#account for much larger data by taking random sample of same size of 2014 dataset
sample1518 = gen_health1518.sample(n = 51858, random_state = 1)
sample1518

120187    4.0
74314     2.0
10751     5.0
34734     2.0
83776     5.0
         ... 
56529     4.0
101909    3.0
36504     2.0
26896     2.0
126199    4.0
Name: genhlth, Length: 51858, dtype: float64

In [50]:
#get counts for each value of general health in 2014 data set
counts14 = getCounts(gen_health14, 5)
counts14

[10706, 18539, 14412, 5803, 2254]

In [51]:
#get counts for each value of general health in 2015-2018 data set sample
counts1518 = getCounts(sample1518,5)
counts1518

[10274, 18518, 14806, 5884, 2257]

In [52]:
#number of non refused or not sure answers in data set
total14 = np.sum(counts14)
total1518 = np.sum(counts1518)

print("2014 total", total14)
print("2015-2018 total",total1518)

2014 total 51714
2015-2018 total 51739


In [53]:
#create combined matrix
genhealth = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], columns = ["Excellent", "Very Good", "Good", "Fair", "Poor"])
genhealth

,Excellent,Very Good,Good,Fair,Poor
2014,10706,18539,14412,5803,2254
2015-2018,10274,18518,14806,5884,2257


In [54]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2014', x = genhealth.columns, y = genhealth.iloc[0]),
                       go.Bar(name = '2015-18', x = genhealth.columns, y = genhealth.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [55]:
#perform chi2 test
test = chi2_contingency(genhealth)
test

(14.77760328295258,
 0.005185422461637733,
 4,
 array([[10487.46503243, 18524.02248364, 14605.46965289,  5842.08788532,
          2254.95494572],
        [10492.53496757, 18532.97751636, 14612.53034711,  5844.91211468,
          2256.04505428]]))

In [56]:
#look at expected values of the data
#it appears that if anything the general health in 2014 is better than 2015-2018 (at least the sample I chose)
df = test[3]

ev = pd.DataFrame(data = df[:,:], index = ["2014", "2015-2018"],
                  columns = ["Excellent", "Very Good", "Good", "Fair", "Poor"]).round(2)

ev

,Excellent,Very Good,Good,Fair,Poor
2014,10487.47,18524.02,14605.47,5842.09,2254.95
2015-2018,10492.53,18532.98,14612.53,5844.91,2256.05


In [57]:
chi2 = test[0]
chi2

14.77760328295258

In [58]:
#note since we have so many data points the p value may be artifically lower
#p value indicates signficant difference?
pvalue = test[1]
pvalue

0.005185422461637733

In [59]:
dof = test[2]
dof

4

In [60]:
#Was there a time you could not see a doctor because of cost?
cost2014 = df_14['medcost']

In [61]:
cost201518 = df_1518['medcost']

In [62]:
#take random sample same size as 2014 data
sample1518 = cost201518.sample(n = 51858, random_state = 1)

In [63]:
#1 corresponds to not being able to see a doctor and 2 is being able to see a doctor
counts14 = getCounts(cost2014, 2)
counts14

[4177, 47546]

In [64]:
counts1518 = getCounts(sample1518, 2)
counts1518

[4097, 47633]

In [65]:
#total number of nonrefused or not sure answers
total14 = np.sum(counts14)
total1518 = np.sum(counts1518)

print("2014 total:", total14)
print("2015-2018 total:",total1518)

2014 total: 51723
2015-2018 total: 51730


In [66]:
#create combined matrix
cost = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], 
                         columns = ["Yes", "No"])
cost

,Yes,No
2014,4177,47546
2015-2018,4097,47633


In [67]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2014', x = cost.columns, y = cost.iloc[0]),
                       go.Bar(name = '2015-18', x = cost.columns, y = cost.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [68]:
#perform chi2 test
test1 = chi2_contingency(cost)
test1

(0.8315231241843959,
 0.361832471245386,
 1,
 array([[ 4136.72007578, 47586.27992422],
        [ 4137.27992422, 47592.72007578]]))

In [69]:
#find contingency matrix for data
df = test1[3]

ev = pd.DataFrame(data = df[:,:], index = ["2014", "2015-2018"],
                  columns = ["Yes","No"]).round(2)

ev

,Yes,No
2014,4136.72,47586.28
2015-2018,4137.28,47592.72


In [70]:
#print summary statistics
#note only 1 degree of freedom, however p value indicates no significant difference
chi2 = test1[0]
pvalue = test1[1]
dof = test1[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 0.8315231241843959 p-value: 0.361832471245386 dof: 1


In [71]:
#splitting cost data on race/income

lowcost14 = low_income14['medcost']
lowcost1518 = low_income1518['medcost']

sample1518 = lowcost1518.sample(n = 11099, random_state = 1)

#get counts for each value of having care or not in 2014 data set
counts14 = getCounts(lowcost14, 2)
total = np.sum(counts14)
#get counts for each value of having care or not in 2014 data set
counts1518 = getCounts(sample1518, 2)
        
lowcost = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], 
                         columns = ["Yes", "No"])
lowcost


,Yes,No
2014,1745,9354
2015-2018,1610,9439


In [72]:
fig = go.Figure(data = [go.Bar(name = '2014', x = cost.columns, y = lowcost.iloc[0]),
                       go.Bar(name = '2015-18', x = cost.columns, y = lowcost.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [73]:
#perform chi2 test for low income interviewees
test2 = chi2_contingency(lowcost)
chi2 = test2[0]
pvalue = test2[1]
dof = test2[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 5.61462247562772 p-value: 0.017811217717322733 dof: 1


In [74]:
#do you have health care coverage?
care14 = df_14['hlthpln1']
care1518 = df_1518['hlthpln1']
sample1518 = care1518.sample(n = 51858, random_state = 1)

#get counts for each value of having care or not in 2014 data set
columns = [1, 2]
counts14 = [0,0]
for row in care14:
    value = (int)(row)
    if value < 7:
        counts14[value-1] += 1

#get counts for each value of having care or not in 2014 data set
columns = [1, 2]
counts1518 = [0,0]
for row in sample1518:
    value = (int)(row)
    if value < 7:
        counts1518[value-1] += 1
        
care = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], 
                         columns = ["Yes", "No"])
care

,Yes,No
2014,48933,2739
2015-2018,49145,2540


In [75]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2014', x = care.columns, y = care.iloc[0]),
                       go.Bar(name = '2015-18', x = care.columns, y = care.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [76]:
#perform chi2 test
test2 = chi2_contingency(care)
pvalue = test2[1]
dof = test2[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 5.61462247562772 p-value: 0.005002025418579685 dof: 1


In [77]:
#do you have health care coverage (with low income participants)?
#splitting coverage data on race/income

lowcare14 = low_income14['hlthpln1']
lowcare1518 = low_income1518['hlthpln1']

sample1518 = lowcare1518.sample(n = 11111, random_state = 1)

#get counts for each value of having care or not in 2014 data set
counts14 = getCounts(lowcare14, 2)

#get counts for each value of having care or not in 2014 data set
counts1518 = getCounts(sample1518, 2)
        
lowcare = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], 
                         columns = ["Yes", "No"])
lowcare


,Yes,No
2014,9952,1159
2015-2018,10014,1045


In [78]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2014', x = lowcare.columns, y = lowcare.iloc[0]),
                       go.Bar(name = '2015-18', x = lowcare.columns, y = lowcare.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [79]:
#perform chi2 test
test2 = chi2_contingency(lowcare)
chi2 = test2[0]
pvalue = test2[1]
dof = test2[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 5.857989598121653 p-value: 0.015506495786049559 dof: 1


In [80]:
#In general how satisfied are you with the health care you recieved?
hc_sat14 = df_14['carercvd']
hc_sat1518 = df_1518['carercvd']
#sample1518 = hc_sat1518.sample(n = 51858, random_state = 1)
print(hc_sat1518)
#get counts for each value of general health in 2014 data set
columns = [1, 2]
counts14 = [0,0,0]
hc_sat14 = hc_sat14.dropna()
for row in hc_sat14:
    value = (int)(row)
    if value < 7:
        counts14[value-1] += 1

#get counts for each value of general health in 2014 data set
columns = [1, 2]
counts1518 = [0,0,0]
hc_sat1518 = hc_sat1518.dropna()
print(hc_sat1518)
for row in hc_sat1518:
    value = (int)(row)
    if value < 7:
        counts1518[value-1] += 1
        
hc_sat = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], 
                         columns = ["Very Satisfied", "Somewhat Satisfied", "Not at all Satisfied"])
hc_sat

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
193644   NaN
193645   NaN
193646   NaN
193647   NaN
193648   NaN
Name: carercvd, Length: 193649, dtype: float64
Series([], Name: carercvd, dtype: float64)


,Very Satisfied,Somewhat Satisfied,Not at all Satisfied
2014,29692,10497,1126
2015-2018,0,0,0


In [81]:
''' uses new data '''

def calculate_low_income(row):
    '''
    returns the following:
        1 = yes, under poverty line
        0 = no, above poverty line
        NaN = unknown (0.40 occurance)
    '''
    if pd.isna(row['income2']):
        return np.NaN # (occurance: 0.006)
    
    if (pd.isna(row['numadult']) or pd.isna(row['children']) or int(row['children']) == 99): #99=refused
        return np.NaN # (occurance: 0.192)
    
    income_grp = int(row['income2'])
    est_income = 0 #Note: est_income is the max household could have (before adjustments)
    income_specs = [10000.0, 15000.0, 20000.0, 25000.0, 35000.0, 50000.0, 75000.0, 75000.0]
    
    if income_grp < 8:
        est_income = income_specs[income_grp - 1]
    elif income_grp == 8: # since 8 is anything more than 75000, this will be unknown
        if (int(row['numadult']) == 1 and int(row['children']) == 88): 
            est_income = income_specs[income_grp - 1] # if 1 adult, no children, then >$75000 earnings for single person (known) 
        else:
            return np.NaN # (occurance: 0.207)
        
    num_adult = row['numadult']
    num_children = row['children']
    
    if (int(num_children) == 88): # 88=none
        num_children = 0
    
    # --------- determine if low income -----------------
    # poverty based on household size as defined by: https://aspe.hhs.gov/2019-poverty-guidelines
    household_size_poverty_line = [12490, 16910, 21330, 25750, 30170, 34590, 39010, 43430]
    above_8_multiplier = 4420
    household_size = int(num_adult + num_children)
        
    if (household_size > 8):
        calculated_line = household_size_poverty_line[7]*2 + above_8_multiplier * (household_size - 8)
        if est_income < calculated_line:
            return 1 
        else: 
            return 0
        
    return est_income < household_size_poverty_line[household_size - 1]*2
    
df_1114['low_income'] = df_1114.apply(lambda row: calculate_low_income(row), axis=1)
df_new_1518['low_income'] = df_new_1518.apply(lambda row: calculate_low_income(row), axis=1)

print("(2011-2014) total low income: ", len(df_1114.query('low_income == 1')))
print("(2011-2014) total unknown: ", df_1114['low_income'].isna().sum())
print("(2011-2014) total households survyed: ", len(df_1114['low_income']))

print("\n(2015-2018) total low income: ", len(df_new_1518.query('low_income == 1')))
print("(2015-2018) total unknown: ", df_new_1518['low_income'].isna().sum())
print("(2015-2018) total households survyed: ", len(df_new_1518['low_income']))

(2011-2014) total low income:  54354
(2011-2014) total unknown:  74354
(2011-2014) total households survyed:  183606

(2015-2018) total low income:  28691
(2015-2018) total unknown:  91849
(2015-2018) total households survyed:  149448


In [82]:
#create new tables for low income
low_income1114 = df_1114.loc[df_1114['low_income'] ==1]
low_income1518 = df_new_1518.loc[df_new_1518['low_income'] ==1]

print(low_income1114.keys())

Index(['xstate', 'genhlth', 'hlthpln1', 'numadult', 'race', 'income2',
       'physhlth', 'poorhlth', 'persdoc2', 'checkup1', 'exerany2', 'sex',
       'educa', 'numadult.1', 'children', 'employ', 'flushot5', 'howlong',
       'lastpap2', 'lastsig3', 'hadsigm3', 'xhcvu651', 'medcost', 'year',
       'medscost', 'drvisits', 'low_income'],
      dtype='object')


In [83]:
#MEDCOST
#splitting cost data on race/income

med_low1114 = low_income1114['medcost']
med_low1518 = low_income1518['medcost']


sample1114 = med_low1114.sample(n = 28691, random_state = 1)

#get counts for each value of having care or not in 2014 data set
counts14 = getCounts(sample1114, 2)
# total14 = np.sum(sample1114)
#get counts for each value of having care or not in 2014 data set
counts1518 = getCounts(med_low1518, 2)
# total15 = np.sum(med_low1518)

# print(total14)
# print(total15)

med_low = pd.DataFrame([counts14, counts1518], index = ["2011-2014", "2015-2018"], 
                         columns = ["Yes", "No"])
med_low

,Yes,No
2011-2014,3517,25027
2015-2018,2270,26270


In [84]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2011-14', x = med_low.columns, y = med_low.iloc[0]),
                       go.Bar(name = '2015-18', x = med_low.columns, y = med_low.iloc[1])])
fig.update_layout(
    barmode='group',
    title="Was there a time in the past 12 months when you<br> needed to see a doctor but could not because of cost?",
    xaxis_title="Response (medcost)",
    yaxis_title="Number of People",
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#7f7f7f"
    )
)
fig.show()

In [85]:
#perform chi2 test
test_medlow = chi2_contingency(med_low)
chi2 = test_medlow[0]
pvalue = test_medlow[1]
dof = test_medlow[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 298.3474351815079 p-value: 7.547669004326359e-67 dof: 1


In [86]:
#HLTHPLN1
#splitting cost data on race/income

hlthpln1114 = low_income1114['hlthpln1']
hlthpln1518 = low_income1518['hlthpln1']

sample1114 = hlthpln1114.sample(n = 28691, random_state = 1)

#get counts for each value of having care or not in 2014 data set
counts1114 = getCounts(sample1114, 2)
# total14 = np.sum(sample1114)
#get counts for each value of having care or not in 2014 data set
counts1518 = getCounts(hlthpln1518, 2)
# total15 = np.sum(med_low1518)

# print(total14)
# print(total15)

hlthpln = pd.DataFrame([counts1114, counts1518], index = ["2011-2014", "2015-2018"], 
                         columns = ["Yes", "No"])
hlthpln

,Yes,No
2011-2014,26118,2388
2015-2018,27291,1189


In [87]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2011-14', x = hlthpln.columns, y = hlthpln.iloc[0]),
                       go.Bar(name = '2015-18', x = hlthpln.columns, y = hlthpln.iloc[1])])
fig.update_layout(
    barmode='group',
    title="Do you have any kind of health care coverage, including health<br> insurance, prepaid plans such as HMOs, or government plans<br> such as Medicare, or Indian Health Service?",
    xaxis_title="Response (hlthpln1)",
    yaxis_title="Number of People",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [88]:
#perform chi2 test
test_hlthpln = chi2_contingency(hlthpln)
chi2 = test_hlthpln[0]
pvalue = test_hlthpln[1]
dof = test_hlthpln[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 426.9376379266439 p-value: 7.540781605356308e-95 dof: 1


In [89]:
#MEDSCOST

#THIS TEST IS NOT AS REPRESENTATIVE

#splitting cost data on race/income

meds_low1114 = low_income1114.loc[low_income1114['medscost'] < 3]
meds_low1518 = low_income1518.loc[low_income1518['medscost'] < 3]

meds_low1114 = meds_low1114['medscost']
meds_low1518 = meds_low1518['medscost']


meds_low1114 = meds_low1114.dropna()
# meds_low1518
meds_low1518 = meds_low1518.dropna()

#meds_low1518

sample1114 = meds_low1114.sample(n = 1091, random_state = 1)

#get counts for each value of having care or not in 2014 data set
counts14 = getCounts(sample1114, 2)
#get counts for each value of having care or not in 2014 data set
counts1518 = getCounts(meds_low1518, 2)

meds_low = pd.DataFrame([counts14, counts1518], index = ["2011-2014", "2015-2018"], 
                         columns = ["Yes", "No"])
meds_low

,Yes,No
2011-2014,110,981
2015-2018,84,970


In [90]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2011-14', x = meds_low.columns, y = meds_low.iloc[0]),
                       go.Bar(name = '2015-18', x = meds_low.columns, y = meds_low.iloc[1])])
fig.update_layout(
    barmode='group',
    title="Not including over-the-counter (OTC) medications, was there a<br> time in the past 12 months when you did not take your <br>medication as prescribed because of cost?",
    xaxis_title="Response (medscost)",
    yaxis_title="Number of People",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [91]:
#perform chi2 test
test_meds_low = chi2_contingency(meds_low)
chi2 = test_meds_low[0]
pvalue = test_meds_low[1]
dof = test_meds_low[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 2.658020223446203 p-value: 0.10302889955331129 dof: 1


In [92]:
#PERSDOC2

#splitting cost data on race/income

doc_low1114 = low_income1114['persdoc2']
doc_low1518 = low_income1518['persdoc2']

doc_low1114 = doc_low1114.dropna()
doc_low1518 = doc_low1518.dropna()


sample1114 = doc_low1114.sample(n = 28691, random_state = 1)

#get counts for each value of having care or not in 2014 data set
counts14 = getCounts(sample1114, 3)
#get counts for each value of having care or not in 2014 data set
counts1518 = getCounts(doc_low1518, 3)

counts14 = [counts14[0]+counts14[1], counts14[2]]
counts1518 = [counts1518[0]+counts1518[1], counts1518[2]]


doc_low = pd.DataFrame([counts14, counts1518], index = ["2011-2014", "2015-2018"], 
                         columns = ["Yes", "No"])
doc_low

,Yes,No
2011-2014,25904,2633
2015-2018,26479,2027


In [93]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2011-14', x = doc_low.columns, y = doc_low.iloc[0]),
                       go.Bar(name = '2015-18', x = doc_low.columns, y = doc_low.iloc[1])])
fig.update_layout(
    barmode='group',
    title="Do you have one person you think of as your personal doctor or <br>health care provider?",
    xaxis_title="Response (persdoc2)",
    yaxis_title="Number of People",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [94]:
#perform chi2 test
test_doc = chi2_contingency(doc_low)
chi2 = test_doc[0]
pvalue = test_doc[1]
dof = test_doc[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 84.81906594085524 p-value: 3.2695631622004007e-20 dof: 1


In [95]:
#CHECKUP1

#PERSDOC2

#splitting cost data on race/income

checkup_low1114 = low_income1114['checkup1']
checkup_low1518 = low_income1518['checkup1']

checkup_low1114 = checkup_low1114.dropna()
checkup_low1518 = checkup_low1518.dropna()


sample1114 = checkup_low1114.sample(n = 28691, random_state = 1)

#get counts for each value of having care or not in 2014 data set
counts14 = getCounts(sample1114, 4)
#get counts for each value of having care or not in 2014 data set
counts1518 = getCounts(checkup_low1518, 4)

checkup_low = pd.DataFrame([counts14, counts1518], index = ["2011-2014", "2015-2018"], 
                         columns = ["less than 12 months ago", "over 1 year but less than 2","over 2 years but less than 5 years", "5 or more years"])
checkup_low

,less than 12 months ago,over 1 year but less than 2,over 2 years but less than 5 years,5 or more years
2011-2014,23140,2760,1251,1012
2015-2018,23965,2331,997,840


In [113]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2011-14', x = checkup_low.columns, y = checkup_low.iloc[0]),
                       go.Bar(name = '2015-18', x = checkup_low.columns, y = checkup_low.iloc[1])])
fig.update_layout(
    barmode='group',
    title="Length of Time Between Last Checkup",
    xaxis_title="Response (checkup1)",
    yaxis_title="Number of People",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [97]:
#perform chi2 test
test_checkup_low = chi2_contingency(checkup_low)
chi2 = test_checkup_low[0]
pvalue = test_checkup_low[1]
dof = test_checkup_low[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)


chi2: 95.25677869586599 p-value: 1.6260384661061595e-20 dof: 3


In [98]:
#test for poor health
poorhlth1114 = low_income1114.loc[low_income1114['poorhlth'] < 77]
poorhlth1518 = low_income1518.loc[low_income1518['poorhlth'] < 77]


poorhlth1114 = poorhlth1114.sample(n = 7333, random_state = 1)

_0_6_1114 = poorhlth1114.loc[(poorhlth1114['poorhlth'] <= 6) & (poorhlth1114['poorhlth'] >= 0)]['poorhlth']
_7_12_1114 = poorhlth1114.loc[(poorhlth1114['poorhlth'] <= 12) & (poorhlth1114['poorhlth'] >= 7)]['poorhlth']
_13_18_1114 = poorhlth1114.loc[(poorhlth1114['poorhlth'] <= 18) & (poorhlth1114['poorhlth'] >= 13)]['poorhlth']
_19_24_1114 = poorhlth1114.loc[(poorhlth1114['poorhlth'] <= 24) & (poorhlth1114['poorhlth'] >= 19)]['poorhlth']
_25_30_1114 = poorhlth1114.loc[(poorhlth1114['poorhlth'] <= 30) & (poorhlth1114['poorhlth'] >= 25)]['poorhlth']


counts1114 = [_0_6_1114.size, _7_12_1114.size, _13_18_1114.size, _19_24_1114.size, _25_30_1114.size]


print(sum(counts1114))

_0_6_1518 = poorhlth1518.loc[(poorhlth1518['poorhlth'] <= 6) & (poorhlth1518['poorhlth'] >= 0)]['poorhlth']
_7_12_1518 = poorhlth1518.loc[(poorhlth1518['poorhlth'] <= 12) & (poorhlth1518['poorhlth'] >= 7)]['poorhlth']
_13_18_1518 = poorhlth1518.loc[(poorhlth1518['poorhlth'] <= 18) & (poorhlth1518['poorhlth'] >= 13)]['poorhlth']
_19_24_1518 = poorhlth1518.loc[(poorhlth1518['poorhlth'] <= 24) & (poorhlth1518['poorhlth'] >= 19)]['poorhlth']
_25_30_1518 = poorhlth1518.loc[(poorhlth1518['poorhlth'] <= 30) & (poorhlth1518['poorhlth'] >= 25)]['poorhlth']

counts1518 = [_0_6_1518.size, _7_12_1518.size, _13_18_1518.size, _19_24_1518.size, _25_30_1518.size]
counts1518
print(sum(counts1518))

poor_health = pd.DataFrame([counts1114, counts1518], index = ["2011-2014", "2015-2018"], 
                         columns = ["0-6 days", "7-12 days", "13-18 days", "19-24 days", "25-30 days"])
poor_health

7333
7333


,0-6 days,7-12 days,13-18 days,19-24 days,25-30 days
2011-2014,2594,1078,883,538,2240
2015-2018,2523,1062,862,590,2296


In [99]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2011-14', x = poor_health.columns, y = poor_health.iloc[0]),
                       go.Bar(name = '2015-18', x = poor_health.columns, y = poor_health.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [100]:
#perform chi2 test
test_poor_hlth = chi2_contingency(poor_health)
chi2 = test_poor_hlth[0]
pvalue = test_poor_hlth[1]
dof = test_poor_hlth[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 4.446016923911332 p-value: 0.34899655299439436 dof: 4


In [101]:
#GENHLTH

#splitting cost data on race/income

gen_hlth1114 = low_income1114.loc[low_income1114['genhlth'] < 7]['genhlth']
gen_hlth1518 = low_income1518.loc[low_income1518['genhlth'] < 7]['genhlth']

gen_hlth1114 = gen_hlth1114.dropna()
# meds_low1518
gen_hlth1518 = gen_hlth1518.dropna()

gen_hlth1518

sample1114 = gen_hlth1114.sample(n = 28564, random_state = 1)

#get counts for each value of having care or not in 2014 data set
counts14 = getCounts(sample1114, 5)
#get counts for each value of having care or not in 2014 data set
counts1518 = getCounts(gen_hlth1518, 5)

gen_hlth = pd.DataFrame([counts14, counts1518], index = ["2011-2014", "2015-2018"], 
                         columns = ["Excellent", "Very Good", "Good", "Fair", "Poor"])
gen_hlth

,Excellent,Very Good,Good,Fair,Poor
2011-2014,3797,7546,9130,5518,2573
2015-2018,3972,8114,9036,5121,2321


In [102]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2011-14', x = gen_hlth.columns, y = gen_hlth.iloc[0]),
                       go.Bar(name = '2015-18', x = gen_hlth.columns, y = gen_hlth.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [103]:
#perform chi2 test
test_gen_hlth = chi2_contingency(gen_hlth)
chi2 = test_gen_hlth[0]
pvalue = test_gen_hlth[1]
dof = test_gen_hlth[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)
#------ DONE WITH NEW TESTS -----------------------

chi2: 52.82029703818233 p-value: 9.292436898390347e-11 dof: 4


In [104]:
''' uses new data '''
def calculate_adjusted_income(row):
    '''
    returns the following:
        0 = less than $5,000 per person,
        1 = less than $10,000 per person, 
        2 = less than $15,000 per person, 
        3 = less than $20,000 per person, 
        4 = less than $25,000 per person, 
        5 = less than $35,000 per person, 
        6 = less than $50,000 per person, 
        7 = less than $75,000 per person, 
        8 = $75,000 or more per person,
        
        NaN (orig 9) = unknown amt per person, because don't know exact incomes of those earning more than $75000 originally. 
        NaN (orig 10) = unknown amt per person, because number of adults / children not given. 
        NaN = unknown because income not given. 
        (9 + 10 + other NaN = 0.40 occurance)
    '''
    if pd.isna(row['income2']):
        return np.NaN # (occurance: 0.006)
    
    if (pd.isna(row['numadult']) or pd.isna(row['children']) or int(row['children']) == 99): #99=refused
        return np.NaN #10 (occurance: 0.192)
    
    income_grp = int(row['income2'])
    est_income = 0 #Note: est_income is the max household could have (before adjustments)
    income_specs = [10000.0, 15000.0, 20000.0, 25000.0, 35000.0, 50000.0, 75000.0]
    
    if income_grp < 8:
        est_income = income_specs[income_grp - 1]
    elif income_grp == 8: # since 8 is anything more than 75000, this will be unknown
        if (int(row['numadult']) == 1 and int(row['children']) == 88): 
            return 8 # if 1 adult, no children, then all earnings for single person. (occurance: 0.02) 
        else:
            #print("high_total_income: adult, children: ", row['numadult'], row['children'])
            return np.NaN #9 (occurance: 0.207)
        
    num_adult = row['numadult']
    num_children = row['children']
    
    if (int(num_children) == 88): # 88=none
        num_children = 0
        
    # --------- calculate adjusted income per person ------------
    
       #print("adults, children: ", num_adult, num_children)
    adj_est_income = est_income / (num_adult + num_children)
    
    new_income_specs = [5000.0, 10000.0, 15000.0, 20000.0, 25000.0, 35000.0, 50000.0, 75000.0]

    for i in range(len(new_income_specs)):
        if adj_est_income <= new_income_specs[i]: 
            return i
    return 8 # Shouldn't get to this point though
    
df_1114['income_per_person'] = df_1114.apply (lambda row: calculate_adjusted_income(row), axis=1)
df_new_1518['income_per_person'] = df_new_1518.apply (lambda row: calculate_adjusted_income(row), axis=1)

#print(df_1114['income_per_person'].dropna())
#print(len(df_1114['income_per_person'])) # ~110163 (60%) final non-NA results
#print(df_1114['income_per_person'].isna().sum() / len(df_1114['income_per_person']))

print("Income Counts Before:")
counts_before_1114 = getCounts(df_1114['income2'].dropna(), 8, 9)
counts_before_1518 = getCounts(df_1518['income2'].dropna(), 8, 9)

income_before_df = pd.DataFrame([counts_before_1114, counts_before_1518], 
                         index = ["2011-14", "2015-18"], 
                         columns = ["<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"]) 
income_before_df

Income Counts Before:


,"<$10,000","<$15,000","<$20,000","<$25,000","<$35,000","<$50,000","<$75,000",">$75,000"
2011-14,7019,8555,11080,13613,15871,21187,24606,53139
2015-18,6070,7934,10297,11797,15259,21784,26003,59590


In [105]:
''' uses new data '''

print("Income adjusted per person:")

counts_per_person_1114 = getCounts(df_1114['income_per_person'].dropna(), 9, 10)
counts_per_person_1518 = getCounts(df_new_1518['income_per_person'].dropna(), 9, 10)

income_per_person_df = pd.DataFrame([counts_per_person_1114, counts_per_person_1518], 
                         index = ["2011-14", "2015-18"], 
                         columns = ["<$5000", "<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"]) 
income_per_person_df

Income adjusted per person:


,<$5000,"<$10,000","<$15,000","<$20,000","<$25,000","<$35,000","<$50,000","<$75,000",">$75,000"
2011-14,11535,11729,13381,14840,5705,14160,5435,5175,27292
2015-18,4194,4831,6050,7179,3235,8093,3132,3390,17495


In [106]:
def highlight_change(current_df, other_df): 
    return pd.DataFrame(np.where(current_df.gt(other_df), 'background-color: lightgreen', 
                                 np.where(current_df.lt(other_df), 'background-color: pink', '')), # else
                        index=current_df.index, 
                        columns=current_df.columns)

In [107]:
''' uses new data '''

# generate table for adjusted income x genhlth for 2011-14, and for 2015-18
income_x_health_1114 = np.zeros(shape=(9, 16)) #income by genhlth
income_x_health_1518 = np.zeros(shape=(9, 16)) #income by genhlth

genhlth_options = [1,2,3,4,5,7,9]

for i in range(9): # for each income group (<$5000, <$10,000, <$15,000, <$20,000, <$25,000, <$35,000, <$50,000, <$75,000, >$75,000)
    for j in range(7): # for each genhlth option (1=excellent through 5=poor, with 7=not sure and 9=refused)
        genhlth_i = genhlth_options[j]
        income_x_health_1114[i, j] = len(df_1114.query('income_per_person == @i & genhlth == @genhlth_i')) 
        income_x_health_1518[i, j] = len(df_new_1518.query('income_per_person == @i & genhlth == @genhlth_i'))
        
    # calculate total respondents
    for opt in range(5):
        income_x_health_1114[i, 7] += income_x_health_1114[i, opt]
        income_x_health_1518[i, 7] += income_x_health_1518[i, opt]
    
    # calculate % that said #
    for opt in range(5):
        income_x_health_1114[i, 8+opt] = (income_x_health_1114[i, opt] / income_x_health_1114[i, 7]).round(4) * 100
        income_x_health_1518[i, 8+opt] = (income_x_health_1518[i, opt] / income_x_health_1518[i, 7]).round(4) * 100

    income_x_health_1114[i, 13] = income_x_health_1114[i, 8] + income_x_health_1114[i, 9] + income_x_health_1114[i, 10]
    income_x_health_1518[i, 13] = income_x_health_1518[i, 8] + income_x_health_1518[i, 9] + income_x_health_1518[i, 10]
    
    income_x_health_1114[i, 14] = income_x_health_1114[i, 11] 
    income_x_health_1518[i, 14] = income_x_health_1518[i, 11]
    
    income_x_health_1114[i, 15] = income_x_health_1114[i, 12] 
    income_x_health_1518[i, 15] = income_x_health_1518[i, 12]
      
income_x_health_1114_df = pd.DataFrame(income_x_health_1114, 
                 index = ["<$5,000", "<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"], 
                 columns=["Excellent", "Very Good", "Good", "Fair", "Poor", "Not Sure", "Refused", "Total Respondents", "% Excellent", "% Very Good", "% Good", "% Fair", "% Poor", "%Positive (1-3)", "%Neutral", "%Negative (5)"])

income_x_health_1518_df = pd.DataFrame(income_x_health_1518, 
                 index = ["<$5,000", "<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"], 
                 columns=["Excellent", "Very Good", "Good", "Fair", "Poor", "Not Sure", "Refused", "Total Respondents", "% Excellent", "% Very Good", "% Good", "% Fair", "% Poor", "%Positive (1-3)", "%Neutral", "%Negative (5)"])

print("Q: General health of the participant (1 = excellent -> 5 = poor) (2011-2014)")
print("Using household income divided by # people in household")
income_x_health_1114_df

Q: General health of the participant (1 = excellent -> 5 = poor) (2011-2014)
Using household income divided by # people in household


,Excellent,Very Good,Good,Fair,Poor,Not Sure,Refused,Total Respondents,% Excellent,% Very Good,% Good,% Fair,% Poor,%Positive (1-3),%Neutral,%Negative (5)
"<$5,000",4503.0,7965.0,8340.0,4477.0,1838.0,95.0,74.0,27123.0,16.60,29.37,30.75,16.51,6.78,76.72,16.51,6.78
"<$10,000",1248.0,2545.0,3674.0,2689.0,1313.0,41.0,25.0,11469.0,10.88,22.19,32.03,23.45,11.45,65.10,23.45,11.45
"<$15,000",1374.0,3089.0,3788.0,2283.0,1148.0,28.0,19.0,11682.0,11.76,26.44,32.43,19.54,9.83,70.63,19.54,9.83
"<$20,000",1782.0,4069.0,4459.0,2154.0,877.0,21.0,19.0,13341.0,13.36,30.50,33.42,16.15,6.57,77.28,16.15,6.57
"<$25,000",2228.0,4935.0,4918.0,2003.0,704.0,24.0,28.0,14788.0,15.07,33.37,33.26,13.54,4.76,81.70,13.54,4.76
"<$35,000",747.0,1886.0,1970.0,822.0,266.0,7.0,7.0,5691.0,13.13,33.14,34.62,14.44,4.67,80.89,14.44,4.67
"<$50,000",2623.0,5544.0,4282.0,1311.0,365.0,15.0,20.0,14125.0,18.57,39.25,30.32,9.28,2.58,88.14,9.28,2.58
"<$75,000",1201.0,2237.0,1489.0,415.0,84.0,4.0,5.0,5426.0,22.13,41.23,27.44,7.65,1.55,90.80,7.65,1.55
">$75,000",1557.0,2076.0,1198.0,268.0,67.0,4.0,5.0,5166.0,30.14,40.19,23.19,5.19,1.30,93.52,5.19,1.30


In [108]:
''' uses new data '''

print("Q: General health of the participant (1 = excellent -> 5 = poor) (Sampled 2015-2018)")
print("Using household income divided by # people in household")
print("Colored based on change from 2011-14 -> green = increase, pink = decrease, white = no change")
 
income_x_health_1518_df.style.apply(highlight_change, other_df=income_x_health_1114_df, axis=None)

Q: General health of the participant (1 = excellent -> 5 = poor) (Sampled 2015-2018)
Using household income divided by # people in household
Colored based on change from 2011-14 -> green = increase, pink = decrease, white = no change


,Excellent,Very Good,Good,Fair,Poor,Not Sure,Refused,Total Respondents,% Excellent,% Very Good,% Good,% Fair,% Poor,%Positive (1-3),%Neutral,%Negative (5)
"<$5,000",2957.000000,5627.000000,5342.000000,2504.000000,970.000000,56.000000,39.000000,17400.000000,16.990000,32.340000,30.700000,14.390000,5.570000,80.030000,14.390000,5.570000
"<$10,000",414.000000,850.000000,1385.000000,1015.000000,520.000000,10.000000,0.000000,4184.000000,9.890000,20.320000,33.100000,24.260000,12.430000,63.310000,24.260000,12.430000
"<$15,000",472.000000,1171.000000,1602.000000,1015.000000,559.000000,8.000000,4.000000,4819.000000,9.790000,24.300000,33.240000,21.060000,11.600000,67.330000,21.060000,11.600000
"<$20,000",686.000000,1836.000000,1990.000000,1082.000000,445.000000,8.000000,3.000000,6039.000000,11.360000,30.400000,32.950000,17.920000,7.370000,74.710000,17.920000,7.370000
"<$25,000",922.000000,2439.000000,2442.000000,1026.000000,341.000000,4.000000,5.000000,7170.000000,12.860000,34.020000,34.060000,14.310000,4.760000,80.940000,14.310000,4.760000
"<$35,000",374.000000,1088.000000,1112.000000,483.000000,168.000000,8.000000,2.000000,3225.000000,11.600000,33.740000,34.480000,14.980000,5.210000,79.820000,14.980000,5.210000
"<$50,000",1365.000000,3123.000000,2576.000000,812.000000,211.000000,4.000000,2.000000,8087.000000,16.880000,38.620000,31.850000,10.040000,2.610000,87.350000,10.040000,2.610000
"<$75,000",532.000000,1316.000000,916.000000,291.000000,73.000000,4.000000,0.000000,3128.000000,17.010000,42.070000,29.280000,9.300000,2.330000,88.360000,9.300000,2.330000
">$75,000",862.000000,1406.000000,842.000000,220.000000,54.000000,3.000000,3.000000,3384.000000,25.470000,41.550000,24.880000,6.500000,1.600000,91.900000,6.500000,1.600000


In [109]:
# generate table for income2 x hcare for 2014, and for 2015-18
print(len(df_1114.index))
print(len(df_new_1518.index))

num_samples = 20
summed_cross_14 = np.zeros(shape=(8, 6))
summed_cross_1518 = np.zeros(shape=(8, 6))
#df_14 = df_1114 # TODO: TEMP for testing, take out later!

for sample_i in range(num_samples): # to average across multiple samples of 2015-2018 data
    sample_df1518 = df_1518.sample(n = 51858)#, random_state = 1)
    #sample_df1518 = df_new_1518 # TODO: TEMP for testing, take out later!

    income_x_hcare_14 = np.zeros(shape=(8, 6)) #income by hlthpln
    income_x_hcare_1518 = np.zeros(shape=(8, 6)) #income by hlthpln

    hlthpln_options = [1,2,7,9]

    for i in range(1,9): # for each income group (<$10,000, <$15,000, <$20,000, <$25,000, <$35,000, <$50,000, <$75,000, >$75,000)
        for j in range(4): # for each healthplan group (Yes, no, not sure, refused)
            hlthpln_i = hlthpln_options[j]
            income_x_hcare_14[i-1, j] = len(df_14.query('income2 == @i & hlthpln1 == @hlthpln_i')) 
            income_x_hcare_1518[i-1, j] = len(sample_df1518.query('income2 == @i & hlthpln1 == @hlthpln_i'))
        
        # calculate total respondents
        income_x_hcare_14[i-1, 4] = income_x_hcare_14[i-1, 0] + income_x_hcare_14[i-1, 1]
        income_x_hcare_1518[i-1, 4] = income_x_hcare_1518[i-1, 0] + income_x_hcare_1518[i-1, 1]
    
        # calculate % that said Yes
        income_x_hcare_14[i-1, 5] = (income_x_hcare_14[i-1, 0] / income_x_hcare_14[i-1, 4]).round(4) * 100
        income_x_hcare_1518[i-1, 5] = (income_x_hcare_1518[i-1, 0] / income_x_hcare_1518[i-1, 4]).round(4) * 100

    summed_cross_14 += income_x_hcare_14
    summed_cross_1518 += income_x_hcare_1518
    
averaged_cross_14 = summed_cross_14 / num_samples
averaged_cross_1518 = summed_cross_1518 / num_samples
    
income_x_hcare_14_df = pd.DataFrame(income_x_hcare_14, 
                 index = ["<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"], 
                 columns=["Yes", "No", "Not Sure", "Refused", "Total Respondents", "Yes Percentage"])

income_x_hcare_1518_df = pd.DataFrame(income_x_hcare_1518, 
                 index = ["<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"], 
                 columns=["Yes", "No", "Not Sure", "Refused", "Total Respondents", "Yes Percentage"])

print("Q: Do you have any health care coverage? (2014)")
income_x_hcare_14_df

183606
149448
Q: Do you have any health care coverage? (2014)


,Yes,No,Not Sure,Refused,Total Respondents,Yes Percentage
"<$10,000",1539.0,224.0,4.0,1.0,1763.0,87.29
"<$15,000",2171.0,224.0,5.0,5.0,2395.0,90.65
"<$20,000",2734.0,330.0,2.0,2.0,3064.0,89.23
"<$25,000",3508.0,381.0,6.0,7.0,3889.0,90.20
"<$35,000",4136.0,342.0,13.0,4.0,4478.0,92.36
"<$50,000",5652.0,373.0,8.0,4.0,6025.0,93.81
"<$75,000",6854.0,229.0,3.0,3.0,7083.0,96.77
">$75,000",14823.0,182.0,12.0,3.0,15005.0,98.79


In [110]:
print("Q: Do you have any health care coverage? (Sampled 2015-2018)")
print("Colored based on change from 2014 -> green = increase, pink = decrease, white = no change")

income_x_hcare_1518_df.style.apply(highlight_change, other_df=income_x_hcare_14_df, axis=None)

#TODO: Percent with health care coverage seems to have increased, check significance / chi-squared?

Q: Do you have any health care coverage? (Sampled 2015-2018)
Colored based on change from 2014 -> green = increase, pink = decrease, white = no change


,Yes,No,Not Sure,Refused,Total Respondents,Yes Percentage
"<$10,000",1478.000000,182.000000,7.000000,2.000000,1660.000000,89.040000
"<$15,000",1913.000000,165.000000,9.000000,6.000000,2078.000000,92.060000
"<$20,000",2393.000000,259.000000,5.000000,4.000000,2652.000000,90.230000
"<$25,000",2825.000000,268.000000,5.000000,2.000000,3093.000000,91.340000
"<$35,000",3789.000000,265.000000,12.000000,3.000000,4054.000000,93.460000
"<$50,000",5549.000000,320.000000,5.000000,3.000000,5869.000000,94.550000
"<$75,000",6895.000000,227.000000,1.000000,2.000000,7122.000000,96.810000
">$75,000",15548.000000,258.000000,6.000000,3.000000,15806.000000,98.370000


In [111]:
# generate table for income1 x genhlth for 2014, and for 2015-18

num_samples = 20

summed_cross_14 = np.zeros(shape=(8, 16))
summed_cross_1518 = np.zeros(shape=(8, 16))

for sample_i in range(num_samples): # to average across multiple samples of 2015-2018 data
    sample_df1518 = df_1518.sample(n = 51858)

    income_x_health_14 = np.zeros(shape=(8, 16)) #income by genhlth
    income_x_health_1518 = np.zeros(shape=(8, 16)) #income by genhlth

    genhlth_options = [1,2,3,4,5,7,9]

    for i in range(1,9): # for each income group (<$10,000, <$15,000, <$20,000, <$25,000, <$35,000, <$50,000, <$75,000, >$75,000)
        for j in range(7): # for each genhlth option (1=excellent through 5=poor, with 7=not sure and 9=refused)
            genhlth_i = genhlth_options[j]
            income_x_health_14[i-1, j] = len(df_14.query('income2 == @i & genhlth == @genhlth_i')) 
            income_x_health_1518[i-1, j] = len(sample_df1518.query('income2 == @i & genhlth == @genhlth_i'))
        
        # calculate total respondents
        for opt in range(5):
            income_x_health_14[i-1, 7] += income_x_health_14[i-1, opt]
            income_x_health_1518[i-1, 7] += income_x_health_1518[i-1, opt]
    
        # calculate % that said #
        for opt in range(5):
            income_x_health_14[i-1, 8+opt] = (income_x_health_14[i-1, opt] / income_x_health_14[i-1, 7]).round(4) * 100
            income_x_health_1518[i-1, 8+opt] = (income_x_health_1518[i-1, opt] / income_x_health_1518[i-1, 7]).round(4) * 100

        income_x_health_14[i-1, 13] = income_x_health_14[i-1, 8] + income_x_health_14[i-1, 9] + income_x_health_14[i-1, 10]
        income_x_health_1518[i-1, 13] = income_x_health_1518[i-1, 8] + income_x_health_1518[i-1, 9] + income_x_health_1518[i-1, 10]
    
        income_x_health_14[i-1, 14] = income_x_health_14[i-1, 11] 
        income_x_health_1518[i-1, 14] = income_x_health_1518[i-1, 11]
    
        income_x_health_14[i-1, 15] = income_x_health_14[i-1, 12] 
        income_x_health_1518[i-1, 15] = income_x_health_1518[i-1, 12]
        
    summed_cross_14 += income_x_health_14
    summed_cross_1518 += income_x_health_1518
    
averaged_cross_14 = summed_cross_14 / num_samples
averaged_cross_1518 = summed_cross_1518 / num_samples
    
income_x_health_14_df = pd.DataFrame(averaged_cross_14, 
                 index = ["<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"], 
                 columns=["Excellent", "Very Good", "Good", "Fair", "Poor", "Not Sure", "Refused", "Total Respondents", "% Excellent", "% Very Good", "% Good", "% Fair", "% Poor", "%Positive (1-3)", "%Neutral", "%Negative (5)"])

income_x_health_1518_df = pd.DataFrame(averaged_cross_1518, 
                 index = ["<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"], 
                 columns=["Excellent", "Very Good", "Good", "Fair", "Poor", "Not Sure", "Refused", "Total Respondents", "% Excellent", "% Very Good", "% Good", "% Fair", "% Poor", "%Positive (1-3)", "%Neutral", "%Negative (5)"])

print("Q: General health of the participant (1 = excellent -> 5 = poor) (2014)")
income_x_health_14_df

Q: General health of the participant (1 = excellent -> 5 = poor) (2014)


,Excellent,Very Good,Good,Fair,Poor,Not Sure,Refused,Total Respondents,% Excellent,% Very Good,% Good,% Fair,% Poor,%Positive (1-3),%Neutral,%Negative (5)
"<$10,000",177.0,314.0,533.0,446.0,287.0,7.0,4.0,1757.0,10.07,17.87,30.34,25.38,16.33,58.28,25.38,16.33
"<$15,000",190.0,437.0,744.0,672.0,353.0,6.0,3.0,2396.0,7.93,18.24,31.05,28.05,14.73,57.22,28.05,14.73
"<$20,000",299.0,685.0,1049.0,685.0,340.0,6.0,4.0,3058.0,9.78,22.40,34.30,22.40,11.12,66.48,22.40,11.12
"<$25,000",455.0,1093.0,1339.0,702.0,298.0,7.0,8.0,3887.0,11.71,28.12,34.45,18.06,7.67,74.28,18.06,7.67
"<$35,000",678.0,1512.0,1504.0,606.0,185.0,6.0,4.0,4485.0,15.12,33.71,33.53,13.51,4.12,82.36,13.51,4.12
"<$50,000",1084.0,2260.0,1915.0,610.0,158.0,5.0,5.0,6027.0,17.99,37.50,31.77,10.12,2.62,87.26,10.12,2.62
"<$75,000",1470.0,3036.0,1942.0,508.0,122.0,4.0,7.0,7078.0,20.77,42.89,27.44,7.18,1.72,91.10,7.18,1.72
">$75,000",4749.0,6530.0,3057.0,541.0,125.0,9.0,9.0,15002.0,31.66,43.53,20.38,3.61,0.83,95.57,3.61,0.83


In [112]:
print("Q: General health of the participant (1 = excellent -> 5 = poor) (Sampled 2015-2018)")
print("Colored based on change from 2014 -> green = increase, pink = decrease, white = no change")
 
income_x_health_1518_df.style.apply(highlight_change, other_df=income_x_health_14_df, axis=None)

Q: General health of the participant (1 = excellent -> 5 = poor) (Sampled 2015-2018)
Colored based on change from 2014 -> green = increase, pink = decrease, white = no change


,Excellent,Very Good,Good,Fair,Poor,Not Sure,Refused,Total Respondents,% Excellent,% Very Good,% Good,% Fair,% Poor,%Positive (1-3),%Neutral,%Negative (5)
"<$10,000",147.400000,274.150000,493.300000,430.500000,265.750000,6.600000,0.850000,1611.100000,9.149500,17.016000,30.611500,26.727000,16.495000,56.777000,26.727000,16.495000
"<$15,000",160.800000,377.500000,720.250000,540.800000,310.950000,2.500000,4.250000,2110.300000,7.619000,17.886000,34.132000,25.625000,14.737500,59.637000,25.625000,14.737500
"<$20,000",281.850000,626.900000,910.400000,639.800000,296.050000,4.250000,2.200000,2755.000000,10.230500,22.754500,33.046000,23.222500,10.748000,66.031000,23.222500,10.748000
"<$25,000",350.000000,877.600000,1131.500000,574.100000,227.100000,2.100000,3.150000,3160.300000,11.076500,27.770000,35.802500,18.161000,7.190000,74.649000,18.161000,7.190000
"<$35,000",539.400000,1352.100000,1377.250000,604.800000,196.650000,3.700000,3.800000,4070.200000,13.252000,33.219500,33.835500,14.859500,4.832500,80.307000,14.859500,4.832500
"<$50,000",1015.350000,2125.550000,1932.600000,600.900000,180.300000,3.550000,2.600000,5854.700000,17.343000,36.303500,33.013000,10.262500,3.080000,86.659500,10.262500,3.080000
"<$75,000",1410.600000,2824.650000,2043.150000,547.650000,135.050000,0.900000,2.750000,6961.100000,20.265500,40.577000,29.351500,7.868000,1.940500,90.194000,7.868000,1.940500
">$75,000",4752.350000,6952.300000,3389.500000,688.900000,143.600000,7.000000,3.400000,15926.650000,29.839000,43.651500,21.282500,4.325500,0.901500,94.773000,4.325500,0.901500
